In [1]:
from sklearn.model_selection import train_test_split
from tensorflow.contrib.layers import fully_connected
from sklearn import datasets
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

In [2]:
circles_data, circles_data_labels = datasets.make_circles(n_samples=50, factor=0.1,
noise=0.1)

In [3]:
circles_data

array([[-0.02777497,  0.01342506],
       [-0.14959601,  0.92249573],
       [ 0.29135892,  0.91149908],
       [-0.99684753,  0.11960708],
       [ 0.09708334,  0.1717598 ],
       [-0.190855  , -0.05338535],
       [-0.66724507, -0.84093401],
       [-0.17325453, -1.00148872],
       [ 0.02938608, -0.26220134],
       [-0.1198195 ,  0.12755392],
       [ 0.76101127, -0.61187836],
       [ 0.60760242,  0.74236439],
       [ 0.13050732,  0.01611388],
       [ 0.1583627 ,  0.06814759],
       [ 0.67627966, -0.8465465 ],
       [ 0.18389709, -0.16003344],
       [ 0.81952436,  0.51946323],
       [-0.75782971,  0.66499073],
       [ 0.20596924, -0.06110788],
       [ 0.93167572,  0.02248229],
       [-0.01338775, -0.13138242],
       [-0.65301065, -0.57937404],
       [-0.01316547,  0.10817302],
       [ 0.17128359, -0.12837106],
       [ 0.02188628,  0.04015171],
       [-0.68561516, -0.59385272],
       [-0.06148146, -0.16652921],
       [-0.14312006,  0.0872595 ],
       [-0.21960778,

In [4]:
circles_data_labels

array([1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 1, 0, 1])

In [5]:
#define network dimension
feat = circles_data.shape[0]
feat_dim = circles_data.shape[1]

#define layer size
hid_layer = 4 #num of hidden layer
out_node =1   #num of output node

In [6]:
input_feat = tf.placeholder(tf.float32, [None,feat_dim], name='input')
value = tf.placeholder(tf.float32,[None,out_node], name='y')

In [7]:
initializer = tf.contrib.layers.xavier_initializer()
hidden1 = fully_connected(input_feat, hid_layer, activation_fn=tf.nn.elu,
                         weights_initializer=initializer)
logits = fully_connected(hidden1, out_node, activation_fn=tf.nn.sigmoid,
                        weights_initializer=initializer)

Instructions for updating:
Colocations handled automatically by placer.


In [8]:
loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=value, logits=logits)
cost = tf.reduce_mean(loss)
learning_rate = 0.01
optimizer = tf.train.AdamOptimizer(learning_rate)
train_op = optimizer.minimize(cost)

In [9]:
# Reshape labels to match placeholder
circles_data_labels = circles_data_labels.reshape(-1, 1)

# Define feed dicts
train_feed = {input_feat: circles_data,
              value: circles_data_labels}
test_feed = {input_feat: circles_data}

# Initialize list to store cost results
iter_cost = []
iters = 1000

# Initialize global variables
init = tf.global_variables_initializer()
# Start session and run loop
with tf.Session() as sess:
    sess.run(init)
    
    # Run training loop
    for i in range(iters):
        _, cost_ = sess.run([train_op, cost],
                       feed_dict=train_feed)
        
        # Append the cost
        iter_cost.append(cost_)
        
    # Plot training loss
    plt.figure(figsize=(10,8))
    plt.plot(iter_cost)
    plt.title("Training Loss")
    plt.xlabel("Number of Iterations")
    plt.ylabel("Mean Cost")
    plt.show()   

    # Make prediction
    pred_prob = sess.run([logits],
                    feed_dict=test_feed)[0]
    
    # Replace probabilities with lables for comparison
    pred_hat = np.where(pred_prob<0.5,0,1)
    # Get prediction accuracy
    acc = np.sum(circles_data_labels.reshape(-1,1)==pred_hat) / len(circles_data_labels)
    print("Test Accuracy %.2f" %acc)
    
    # Plot the decision boundary
    # Determine grid range in x and y directions
    feat_min, feat_max = circles_data[:, 0].min()-0.1, circles_data[:, 0].max()+0.1
    pred_min, pred_max = circles_data[:, 1].min()-0.1, circles_data[:, 1].max()+0.1
    
    # Set grid spacing parameter
    spacing = min(feat_max - feat_min, pred_max - pred_min) / 100

    # Create grid
    XX, YY = np.meshgrid(np.arange(feat_min, feat_max, spacing),
                   np.arange(pred_min, pred_max, spacing))
    
    # Concatenate data to match input
    data = np.hstack((XX.ravel().reshape(-1,1), 
                      YY.ravel().reshape(-1,1)))

    # Pass data to predict method
    db_prob = sess.run([logits],
                      feed_dict={input_feat: data})[0]
    clf = np.where(db_prob<0.5,0,1)
    
    Z = clf.reshape(XX.shape)

    plt.figure(figsize=(10,8))
    plt.contourf(XX, YY, Z, cmap=plt.cm.Spectral, alpha=0.8)
    plt.scatter(circles_data[:,0], circles_data[:,1], c=circles_data_labels, 
                cmap=plt.cm.Spectral)
    plt.show()

InternalError: Blas GEMM launch failed : a.shape=(50, 2), b.shape=(2, 4), m=50, n=4, k=2
	 [[node fully_connected/MatMul (defined at <ipython-input-7-ca95e5062e9b>:3) ]]
	 [[node Mean (defined at <ipython-input-8-5aa89af35f79>:2) ]]

Caused by op 'fully_connected/MatMul', defined at:
  File "/home/jeff/anaconda3/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/jeff/anaconda3/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/jeff/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/jeff/anaconda3/lib/python3.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/jeff/anaconda3/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/home/jeff/anaconda3/lib/python3.7/site-packages/tornado/platform/asyncio.py", line 148, in star